In [15]:
import os
import cv2
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import warnings

warnings.filterwarnings(action='ignore') 

def get_blur(flag, img_dir, label_dir, save_path='null'):
    # 파일 읽어들이기
    imgs = os.listdir(img_dir)
    imgs = [file for file in imgs if os.path.splitext(file)[1] == '.jpg']
    labels  = os.listdir(label_dir)
    labels = [file for file in labels if file != 'classes.txt']
    
    # 파일만큼 반복문
    for flag in range(4):
        for idx in range(len(imgs)):
            image = cv2.imread(img_dir + imgs[idx])
    #         print(imgs[idx])

            x = image.shape[1]
            y = image.shape[0]

            file = open(label_dir + labels[idx], mode="r")
            point = file.read().split(' ')
            point = [float(p) for p in point]
            file.close()

            # 원래 좌표 입력
            bbs = BoundingBoxesOnImage([
                BoundingBox(x1=(point[1])*x, y1=(point[2])*y, x2=(point[1]+point[3])*x, y2=(point[2]+point[4])*y),
            ], shape=image.shape)
            # 필터 설정
            seq, fil_name = select_filter(flag)
            seq_det = seq.to_deterministic()

            # flip 처리 이후 이미지, 박싱
            image_aug = seq_det.augment_images([image])[0]
            bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
    #         image_before = bbs.draw_on_image(image, thickness=20)
    #         image_after = bbs_aug.draw_on_image(image_aug, thickness=20, color=[0, 0, 255])

    #         after = bbs_aug.bounding_boxes[0]

            # 저장
            save_data(save_path, image_aug, point, fil_name, imgs[idx])
        
def save_data(save_path, image_aug, point, fil_name, imgs):
    # 파일명 정리
    splits = imgs.split('.')
    file_name = splits[0]
    file_name = fil_name + '_' + file_name 
    ext = '.' + splits[1]
    # 저장위치 설정
    save_path = save_path + fil_name +'/'
    chk_dir(save_path)
    
    # 이미지 저장
    cv2.imwrite(save_path + file_name + ext, image_aug)
    print(save_path + file_name + ext)
    # 라벨링 txt 저장
    f = open(save_path + file_name + ".txt", 'w')
    f.write('0 {} {} {} {}'.format(point[1], point[2], point[3], point[4]))
    f.close()
    
def select_filter(flag):
    fil_name = ''
    seq = iaa.Sequential([])
    if flag == 0:
        seq = iaa.Fliplr(1.0)
        fil_name = 'Fliplr'
    elif flag == 1:
        seq = iaa.ShearX((-20, 20))
        fil_name = 'ShearX'
    elif flag == 2:
        seq = iaa.imgcorruptlike.MotionBlur(severity=2)
        fil_name = 'MotionBlur'
    elif flag == 3:
        iaa.CoarseDropout(0.03, size_percent=0.15)
        fil_name = 'CoarseDropout'
    return seq, fil_name

def chk_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print ('Error: Creating directory. ' +  path)

In [17]:
# img_dir = ['./data/Multi-view_tracking_indoor/00001_market/frames/02/',
#           './data/Multi-view_tracking_indoor/06275_market/frames/01/',
#           './data/Multi-view_tracking_outdoor/31293_street/frames/02/',
#           './data/Multi-view_tracking_outdoor/30012_street/frames/01/']
img_dir = './data/Multi-view_tracking_outdoor/30009_street/frames/01/'

# label_dir = ['./labels_txt/Multi-view_tracking_indoor/00001_market/json/1-5/02/',
#             './labels_txt/Multi-view_tracking_indoor/06275_market/json/1-5/01/',
#             './labels_txt/Multi-view_tracking_outdoor/31293_street/json/1-5/02/',
#             './labels_txt/Multi-view_tracking_outdoor/30012_street/json/1-5/01/']
label_dir = './labels_txt/Multi-view_tracking_outdoor/30009_street/json/1-5/01/'

# save_path = ['./train/raw/00001_market/',
#             './train/raw/06275_market/',
#             './train/raw/31293_street/',
#             './train/raw/30012_street/']
save_path = './train/raw/30009_street/'
fil_name = 'CoarseDropout/'

# for i in range(4):
#     get_blur(0, img_dir[i], label_dir[i], save_path[i]+fil_name)
get_blur(img_dir, label_dir, save_path)

./train/raw/30009_street/Fliplr/Fliplr_00005.jpg
./train/raw/30009_street/Fliplr/Fliplr_00006.jpg
./train/raw/30009_street/Fliplr/Fliplr_00007.jpg
./train/raw/30009_street/Fliplr/Fliplr_00008.jpg
./train/raw/30009_street/Fliplr/Fliplr_00009.jpg
./train/raw/30009_street/Fliplr/Fliplr_00010.jpg
./train/raw/30009_street/Fliplr/Fliplr_00011.jpg
./train/raw/30009_street/Fliplr/Fliplr_00012.jpg
./train/raw/30009_street/Fliplr/Fliplr_00013.jpg
./train/raw/30009_street/Fliplr/Fliplr_00014.jpg
./train/raw/30009_street/Fliplr/Fliplr_00015.jpg
./train/raw/30009_street/Fliplr/Fliplr_00016.jpg
./train/raw/30009_street/Fliplr/Fliplr_00017.jpg
./train/raw/30009_street/Fliplr/Fliplr_00018.jpg
./train/raw/30009_street/Fliplr/Fliplr_00019.jpg
./train/raw/30009_street/Fliplr/Fliplr_00020.jpg
./train/raw/30009_street/ShearX/ShearX_00005.jpg
./train/raw/30009_street/ShearX/ShearX_00006.jpg
./train/raw/30009_street/ShearX/ShearX_00007.jpg
./train/raw/30009_street/ShearX/ShearX_00008.jpg
./train/raw/30009_st